<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/LCEL%2C%20Runnables%20y%20cadenas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. LCEL, LangChain Expression Language**
---

LCEL se introdujo en Langchain **a mediados de 2023**, específicamente con la **versión 0.0.142**, lanzada el **19 de julio de 2023**

La introducción de **LCEL** en Langchain fue una respuesta a la necesidad de una forma más potente, flexible y fácil de usar para construir aplicaciones de lenguaje complejas. Proporcionó una sintaxis declarativa, mejoró la legibilidad, facilitó la depuración y habilitó funcionalidades avanzadas como el streaming y la ejecución asíncrona, consolidándose como una pieza fundamental del ecosistema de Langchain.

**LCEL se basa en la Composición, no en Tipos Predefinidos:**

-   **Operador Pipe (|):** La piedra angular de LCEL es el operador pipe. Este operador te permite encadenar componentes de forma secuencial, enviando la salida de un componente como entrada al siguiente. Esto es inherentemente secuencial, pero no se define como un tipo de cadena "secuencial".
    
-   **Primitivas Runnable:** LCEL se basa en la interfaz Runnable. Cualquier objeto que implemente esta interfaz puede ser parte de una cadena LCEL. Esto incluye modelos de lenguaje, prompts, parsers, retrievers, etc.
    
-   **Flexibilidad Total:** La clave es que puedes combinar estas primitivas Runnable de cualquier manera que tenga sentido para tu aplicación. No estás limitado a estructuras predefinidas.

**Ventajas de este enfoque:**

-   **Mayor Flexibilidad:** No estás limitado por las estructuras predefinidas. Puedes crear flujos de trabajo exactamente como los necesitas.
    
-   **Reutilización de Componentes:** Los componentes individuales pueden ser reutilizados en diferentes cadenas con diferentes flujos de trabajo.
    
-   **Claridad y Composición:** El uso del operador pipe hace que la lógica de la cadena sea más clara y fácil de entender.
    
-   **Optimización:** La forma en que construyes la cadena influye en cómo se puede optimizar su ejecución (por ejemplo, para paralelismo).
    

**En resumen, LCEL te proporciona las herramientas y la sintaxis para orquestar tus flujos de trabajo de manera flexible y poderosa. En lugar de imponer tipos de cadenas predefinidos, te da la libertad de construir las cadenas que mejor se adapten a tus necesidades, implementando patrones secuenciales, condicionales o paralelos según sea necesario.**

Y todo esta abstración se basa en los **Runnables**

# **2. Definición del Concepto Runnable**
---

En esencia, un **Runnable** en LangChain es una **unidad básica y ejecutable** dentro de LCEL. Piensa en ellos como los bloques de construcción fundamentales con los que puedes crear flujos de trabajo complejos. Cada Runnable toma una entrada, realiza alguna operación sobre ella y produce una salida.

**Analogía:** Imagina una cadena de montaje en una fábrica. Cada estación de trabajo (o máquina) realiza una tarea específica sobre la pieza que llega y la pasa a la siguiente estación. En LangChain, cada Runnable sería una de esas estaciones de trabajo.  


## **Puntos Clave:**

- **Ejecutables:** Los Runnables pueden ser "ejecutados" para procesar datos.  

- **Componibles:** La verdadera potencia de los Runnables reside en su capacidad
-**Abstractos:**  La interfaz `Runnable` define un contrato común, pero la implementación específica de cada Runnable puede variar enormemente (desde llamar a un modelo de lenguaje hasta realizar una simple transformación de datos).

## **Interfaz Estándar de Runnables**

Una de las grandes ventajas de los Runnables es que comparten una **interfaz estándar**. Esto significa que, una vez que entiendes cómo interactuar con un Runnable, puedes aplicar ese conocimiento a cualquier otro Runnable, ¡independientemente de lo que haga internamente!

La interfaz principal se basa en una serie de métodos que permiten ejecutar el Runnable de diferentes maneras.




# **Preparando el entorno del cuaderno**
---
Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).

- Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google Hugging Face, Mistral, Together y Anthropic

- Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

- Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.
El uso de las API de OpenAI y Anthropic es de pago. El resto son gratuitas y para usarlas basta con registrase y generar una API Key.  

En el primer cuaderno encontraras los enlaces a estos servicios y este codigo explicado

In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic


# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

#**3. Métodos Principales**
---

Vamos a explorar los métodos más importantes que encontrarás en la interfaz de un Runnable:



## **3.1 `.invoke()`**

El método `.invoke()` es la forma más básica de ejecutar un Runnable. Toma una **única entrada** y devuelve la **salida** del Runnable después de procesar esa entrada.

Importamos `RunnableLambda`, una forma sencilla de crear un Runnable a partir de una función Python (lo veremos en detalle más adelante).
 - Definimos un Runnable que toma una cadena (`x`) y devuelve su versión en mayúsculas.  
 - Llamamos al método `.invoke()` pasándole la cadena "hola mundo" como entrada.
 - La variable `resultado` contiene la salida del Runnable, que en este caso es "HOLA MUNDO".  
  
<br>


 *¡Inténtalo tú!* Modifica el código anterior para que el Runnable convierta el texto a minúsculas.

In [ ]:
from langchain_core.runnables import RunnableLambda

# Creamos un Runnable sencillo que convierte el texto a mayúsculas
mi_runnable = RunnableLambda(lambda x: x.upper())

# Ejecutamos el Runnable con el método invoke()
resultado = mi_runnable.invoke("hola, mundo")

print(f"Resultado de invoke(): {resultado}")

## **3.2 `.batch()`**

El método `.batch()` te permite ejecutar un Runnable con **múltiples entradas al mismo tiempo**. Es muy útil cuando tienes una lista de datos que quieres procesar de manera eficiente. Devuelve una **lista de salidas**, donde cada salida corresponde a la entrada en la misma posición.


*   Utilizamos el mismo Runnable de antes.
*   Creamos una lista de cadenas llamada `entradas`.
*   Llamamos a `.batch()` pasándole la lista de entradas.
*   La variable `resultados` contiene una lista con las versiones en mayúsculas de cada palabra en la lista `entradas`.


In [ ]:
from langchain_core.runnables import RunnableLambda

# El mismo Runnable que convierte a mayúsculas
mi_runnable = RunnableLambda(lambda x: x.upper())

# Una lista de entradas
entradas = ["manzana", "banana", "cereza"]

# Ejecutamos el Runnable con el método batch()
resultados = mi_runnable.batch(entradas)

print(f"Resultados de batch(): {resultados}")

## **📝Desafío:**
Crea un Runnable que calcule la longitud de una cadena y luego usa `.batch()` para obtener la longitud de las palabras en la lista `entradas`.

In [ ]:
# Escribe tu solución aquí

## 👀 Solución:

In [ ]:
from langchain_core.runnables import RunnableLambda

# Crea un Runnable que calcule la longitud de una cadena
longitud_runnable = RunnableLambda(lambda x: len(x))

entradas = ["manzana", "banana", "cereza"]

# Usa .batch() para obtener las longitudes
longitudes = longitud_runnable.batch(entradas)

print(f"Longitudes de batch(): {longitudes}")

## **3.3 `.stream()`**

El método `.stream()` es diferente a los anteriores. En lugar de devolver toda la salida de una vez, `.stream()` devuelve un **generador**. Esto significa que puedes ir recibiendo las salidas del Runnable **a medida que se van generando**, lo que es especialmente útil para procesar grandes cantidades de datos o para aplicaciones que necesitan mostrar resultados en tiempo real.

- Creamos un Runnable que añade un saludo a la entrada.  

- Llamamos a `.stream()` con una lista de nombres.  

- El resultado es un generador. Usamos un bucle `for` para iterar sobre el generador y obtener cada salida individualmente.

**Puntos Importantes sobre `.stream()`:**

- **Eficiencia de Memoria:** No carga todas las salidas en memoria a la vez.  

- **Procesamiento en Tiempo Real:** Permite mostrar resultados a medida que se generan.  

- **Iteración:** Debes iterar sobre el generador para obtener las salidas.


In [ ]:
from langchain_core.runnables import RunnableLambda

# Un Runnable que añade un saludo a cada palabra
saludo_runnable = RunnableLambda(lambda x: f"¡Hola, {x}!")

entradas = ["Ana", "Beto", "Carla"]

# Obtenemos un generador usando stream()
flujo = saludo_runnable.stream(entradas)

print("Resultados de stream():")
for saludo in flujo:
    print(saludo)

En la interacción con modelos de lenguaje como GPT, el streaming ofrece una alternativa a la espera de la respuesta completa. En lugar de una entrega única, el modelo emite la respuesta progresivamente, token por token (fragmentos de palabras o caracteres). Esta transmisión gradual mejora la experiencia del usuario, quien percibe una respuesta más rápida y natural al ver el texto aparecer en tiempo real. Además, el streaming permite iniciar el procesamiento de la respuesta en tiempo real, sin esperar a que el modelo termine de generar toda la salida. En esencia, el streaming transforma la interacción con el LLM de una espera prolongada a una recepción continua y dinámica.

In [ ]:
llm_groq = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

# Obtenemos un "generador"
respuesta = llm_groq.stream("Explica los verbos modales en inglés")

respuesta

In [ ]:
#iteramos el generador para mostrar la respuesta
for token in respuesta:
    print(token.content, end="", flush=True)

## **3.4 `ainvoke()`, `abatch()`, `astream()`  **

Hasta ahora, hemos visto métodos síncronos, lo que significa que la ejecución del programa se pausa hasta que el Runnable termina de procesar la entrada.

LangChain también proporciona versiones **asíncronas** de estos métodos, que permiten realizar otras tareas mientras el Runnable está en ejecución, mejorando la eficiencia en aplicaciones concurrentes.

- El método `.ainvoke()` es la versión asíncrona de `.invoke()`.   
- El método `.abatch()` es la versión asíncrona de `.batch()`.   
- El método `.astream()` es la versión asíncrona de `.stream()`.

Para usarlos, necesitas utilizar la sintaxis `async/await` de Python.

Los trataremos más adelante.


# **4. Referencias**
---
1. https://python.langchain.com/docs/concepts/runnables/

2. https://python.langchain.com/api_reference/core/runnables.html
3. https://python.langchain.com/docs/how_to/lcel_cheatsheet/

4. https://dzone.com/articles/guide-to-langchain-runnable-architecture

5. https://medium.com/@danushidk507/runnables-in-langchain-e6bfb7b9c0ca

6. https://www.youtube.com/watch?v=8aUYzb1aYDU

7. https://medium.com/@james.li/mental-model-to-building-chains-with-langchain-expression-language-lcel-with-branching-and-36f185134eac

8. https://medium.com/@ulrichw/list/langchain-lcel-85af4f4ff883

9. https://medium.com/@anuragmishra_27746/practical-hands-on-with-langchain-expression-language-lcel-for-building-langchain-agent-chain-2a9364dc4ca3

10. https://www.pinecone.io/learn/series/langchain/langchain-expression-language/